Les données sont dispo à cette url :

http://alt.qcri.org/semeval2016/task5/index.php?id=data-and-tools

In [58]:
!pip install pandas_read_xml

     |████████████████████████████████| 15.1 MB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 42.0 MB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 7.2 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.1
    Uninstalling numpy-1.18.1:
      Successfully uninstalled numpy-1.18.1
  Attempting uninstall: pytz
    Found existing installation: pytz 2019.3
    Uninstalling pytz-2019.3:
      Successfully uninstalled pytz-2019.3
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.8
    Uninstalling urllib3-1.25.8:
      Successfully uninstalled urllib3-1.25.8


In [3]:
import pandas_read_xml as pdx
import numpy as np

In [11]:
df = pdx.read_xml(
    "datasets/ABSA16FR_Restaurants_Train-withcontent.xml",
    ['Reviews']
)

Les Données contiennent plus d'information que nécessaire : chaque commentaire est séparé en phrase qui ont elles même plusieurs notes chacune (service, nourriture, prix ...). Pour simplifier le problème, on va prendre la moyenne des notes et garder une note finale :
- `'negative'` (0)
- `'neutral'` (1)
- `'positive'` (2)

In [12]:
def extract_text(row):
    
    sent = row['sentences']['sentence']
    
    if isinstance(sent, list):
        text = '. '.join([s['text'] for s in sent if s['text'] is not None])
    else:
        text = sent['text']

    return text

In [13]:
def extract_label(row, binary=True):
    
    sent = row['sentences']['sentence']
    
    def _extract_label(r, binary=True):
        
        if binary:
            correspondance_labels = {
                'negative' : 0,
                'neutral' : .5,
                'positive' : 1
            }
        else:
            correspondance_labels = {
                'negative' : 0,
                'neutral' : 1,
                'positive' : 2
            }
        
        opinions = r['Opinions']['Opinion']
        
        if isinstance(opinions, list):
            polarities = [o['@polarity'] for o in opinions]
            pol_int = [correspondance_labels[p] for p in polarities]

            label = int(np.round(np.mean(pol_int)))
        
        else:
            label = correspondance_labels[opinions['@polarity']]
            
        return label
    
    if isinstance(sent, list):
        labels = [x for x in sent if x is not None]
        labels = [x for x in labels if x['Opinions'] is not None]
        labels = [_extract_label(s) for s in labels]
        label = int(np.round(np.mean(labels)))
    else:
        label = _extract_label(sent)
    
    return label

In [14]:
df['texte'] = df.Review.apply(extract_text)
df['label_2'] = df.Review.apply(extract_label, binary=True)
df['label_3'] = df.Review.apply(extract_label, binary=False)

In [15]:
df.texte[100]

'Service de tres bonne qualité, restaurant agreable et cuisine delicieuse !. Les produits sont frais et de saison, cest un regal !. Jy retournerai a coup sur pour decouvrir les autres plats de la carte.'

In [21]:
df.label_2[100]

1

In [22]:
df.drop('Review', axis=1).to_csv("datasets/restaurants.csv")